In [2]:
import numpy as np

In [3]:
np.random.seed(0)
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
]
P = np.array(P)
rewards = [-1, -2, -2, 10, 1, 0]
gamma = 0.5

def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = rewards[chain[i] - 1] + gamma * G
    return G

chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print(f"Return G for the chain {chain} starting at index {start_index}: {G}")


Return G for the chain [1, 2, 3, 6] starting at index 0: -2.5


In [4]:
def compute(P, rewards, gamma, states_num):
    rewards = np.array(rewards).reshape((-1, 1))
    value = np.dot(np.linalg.inv(np.eye(states_num) - gamma * P), rewards)
    return value

V = compute(P, rewards, gamma, len(P))
print("Each state's value of MRP\n", V)

Each state's value of MRP
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


In [5]:
# State Set
S = ["s1", "s2", "s3", "s4", "s5"]

# Action Set
A = ["stay in s1", "go to s1", "go to s2", "go to s3", "go to s4", "go to s5", "probably go to s2/s3/s4"]

# State Transition Function, P(s'|s,a)
P = {
    "s1-stay in s1-s1": 1.0,
    "s1-go to s2-s2": 1.0,
    "s2-go to s1-s1": 1.0,
    "s2-go to s3-s3": 1.0,
    "s3-go to s4-s4": 1.0,
    "s3-go to s5-s5": 1.0,
    "s4-go to s5-s5": 1.0,
    "s4-probably go to s2/s3/s4-s2": 0.2,
    "s4-probably go to s2/s3/s4-s3": 0.4,
    "s4-probably go to s2/s3/s4-s4": 0.4,
}

# Reward Function, R(s,a)
R = {
    "s1-stay in s1": -1,
    "s1-go to s2": 0,
    "s2-go to s1": -1,
    "s2-go to s3": -2,
    "s3-go to s4": -2,
    "s3-go to s5": 0,
    "s4-go to s5": 10,
    "s4-probably go to s2/s3/s4": 1
}

# Discount Factor
gamma = 0.5

MDP = (S, A, P, R, gamma)

# Policy 1, random policy
Pi_1 = {
    "s1-stay in s1": 0.5,
    "s1-go to s2": 0.5,
    "s2-go to s1": 0.5,
    "s2-go to s3": 0.5,
    "s3-go to s4": 0.5,
    "s3-go to s5": 0.5,
    "s4-go to s5": 0.5,
    "s4-probably go to s2/s3/s4": 0.5
}

# Policy 2
Pi_2 = {
    "s1-stay in s1": 0.6,
    "s1-go to s2": 0.4,
    "s2-go to s1": 0.3,
    "s2-go to s3": 0.7,
    "s3-go to s4": 0.5,
    "s3-go to s5": 0.5,
    "s4-go to s5": 0.1,
    "s4-probably go to s2/s3/s4": 0.9
}

# connect two strings with '-' to define the P-R variables
def join(str1, str2):
    return str1 + '-' + str2


In [6]:
gamma = 0.5
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0]
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0.0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, len(P_from_mdp_to_mrp))
print("Each state value of MDP\n", V)

Each state value of MDP
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


In [7]:
def sample(MDP, Pi, timestep_max, number):
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0)
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            
            rand, temp = np.random.rand(), 0
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            
            episode.append((s, a, r, s_next))
            s = s_next
        episodes.append(episode)
    return episodes

In [8]:
episodes = sample(MDP, Pi_1, 20, 5)
print('Episode 1\n', episodes[0])
print('Episode 2\n', episodes[1])
print('Episode 5\n', episodes[4])

Episode 1
 [('s1', 'go to s2', 0, 's2'), ('s2', 'go to s3', -2, 's3'), ('s3', 'go to s5', 0, 's5')]
Episode 2
 [('s4', 'probably go to s2/s3/s4', 1, 's4'), ('s4', 'go to s5', 10, 's5')]
Episode 5
 [('s2', 'go to s3', -2, 's3'), ('s3', 'go to s4', -2, 's4'), ('s4', 'go to s5', 10, 's5')]


In [9]:
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]
        

In [10]:
timestep_max = 20
episodes = sample(MDP, Pi_1, timestep_max, 1000)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("state value of MDP with MC\n", V)

state value of MDP with MC
 {'s1': -1.228923788722258, 's2': -1.6955696284402704, 's3': 0.4823809701532294, 's4': 5.967514743019431, 's5': 0}


In [13]:
def occupancy(episodes, s, a, timestep_max, gamma):
    """Count the number of times (s,a) is visited in the episode to compute the occupancy measure."""
    rho = 0
    total_times = np.zeros(timestep_max)
    occur_times = np.zeros(timestep_max)
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            if s == s_opt and a == a_opt:
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if total_times[i]:
            rho += (gamma ** i) * occur_times[i] / total_times[i]
    return (1 - gamma) * rho

In [14]:
gamma = 0.5
timestep_max = 1000
episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "probably go to s2/s3/s4", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "probably go to s2/s3/s4", timestep_max, gamma)
print(rho_1, rho_2)

0.11365220293209718 0.2487246201875158
